In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from bs4 import BeautifulSoup
import re
from uuid import uuid4

In [3]:
class Crawler:

    def get_html(self):
        pass

    def get_card_info(self):
        pass

    def get_links(self, page):
        pass

In [41]:
class CrawlerEda(Crawler):

    def get_html(self, page):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        url = f"https://eda.ru/recepty?page={page}"
        #browser.implicitly_wait(30)
        browser.get(url)
        wait = WebDriverWait(browser, 10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        return soup

    def get_card_info(self, link):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        action = ActionChains(browser)
        count = 0

        #linker = linkers[number]['link']
        #browser.get(linker)
        browser.get(link)

        wait = WebDriverWait(browser, 10)
        browser.implicitly_wait(30)

        soup_card = BeautifulSoup(browser.page_source, 'html.parser')
        return soup_card

    def get_soup(self, url):
        return BeautifulSoup(requests.get(url).text, "html.parser")

    def get_links(self,first_page, last_page):
        links = []
        for i in range(first_page, last_page):
            html = CrawlerEda().get_html(i)
            links.extend(ParserEda(html).get_cards())
            #print(links)
        return links

In [5]:
class Parser:

    def __init__(self, html):
        self.html = html

    def get_cards(self):
        pass

    def __get_ingredients(self):
        pass

    def __get_calories(self):
        pass

    def __get_manual(self):
        pass

    def __get_category(self):
        pass

    def get_recipe(self):
        pass

In [137]:
class ParserEda(Parser):

    def get_cards(self):
        linkers = []
        for recipe in self.html.findAll('div', class_='emotion-1f6ych6'):
            try:
                link = "https://eda.ru"+recipe.find('a', class_='emotion-12sjte8').get('href')
                photo = str(recipe.find('img', class_='emotion-1bi124x').get('src')).replace('c180x180', 'c500x500')
                #print(link)
                #print(photo)
                linkers.append(Link(link, photo))
            except AttributeError:
                print("There's no item with that code")
        #print(len(linkers))
        return linkers

    def get_recipe(self):
        print("get_recipe")

        parser = ParserEda(self.html)
        card = self.html.find('div', class_='emotion-2k9cfu')

        name_dish = card.find('div', class_='emotion-19rdt1j').find('h1', class_='emotion-gl52ge').text
        print(name_dish)

        count_portion = str(re.sub(r'\bпорц[ий|ия|ии]*\b', '', card.find('div', class_='emotion-1047m5l').text).strip())
        cook_time = card.find('div', class_='emotion-18q6z5o').find('div', class_='emotion-my9yfq').text
        print(cook_time)

        manual = parser.__get_manual()

        recipe = Recipe(name_dish, count_portion, cook_time, manual)

        #time.sleep(2)

        helper = DBHelper()
        helper.create_table_simple('recipe', ['name_dish', 'count_portion', 'cook_time', 'manual'])
        helper.insert('recipe', ['name_dish', 'count_portion', 'cook_time', 'manual'],
                                [recipe.name, recipe.count_portion, recipe.cook_time, recipe.manual ])

        #self, table_name, column_args, arg
        #находим айдишник нужного рецепта
        recipe_ = helper.get('recipe', ['name_dish', 'count_portion'], [name_dish, count_portion])
        id_ = recipe_[0][0]

        categories = parser.__get_categories(id_)

        ingredients = parser.__get_ingredients(id_)

        calories = parser.__get_calories(id_)

        #name_dish, count_portion, cook_time, ingredients, categories, manual

    #autoincrement
    def __get_ingredients(self, id):
        all_ingredients = {}
        name_ingredients = []

        for ing in self.html.findAll('div', class_='emotion-bjn8wh'):
            name_ingredients.append(ing.text)

        for i in range (len(name_ingredients)):
            all_ingredients[name_ingredients[i]] = self.html.findAll('span', class_='emotion-bsdd3p')[i].text

        helper = DBHelper()
        helper.create_table_simple('ingredient', ['name_ingr']) #autoincrement
        #table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params
        helper.create_table('recipe_ingredients', 'id', 'recipe', 'recipe_id', 'id', ['recipe_id', 'ingr_id', 'count'], True) #

        for name in name_ingredients:
            helper.insert('ingredient', ['name_ingr'], [name])

        for i in range (len(name_ingredients)):
            #self, table_name, column_args, args
            ingr_ = helper.get('ingredient', ['name_ingr'], [name_ingredients[i]])
            helper.insert('recipe_ingredients', ['recipe_id', 'ingr_id', 'count'], [id, ingr_[0][0], all_ingredients[ingr_[0][1]]])

        return all_ingredients

    #autoincrement
    def __get_categories(self, id):
        categories = []

        #дурацкие категории
        bad_categories = ['Главная', 'Проверено «Едой»', 'Пошаговые рецепты', 'Видеорецепты', 'Рецепты с историей']

        for category in self.html.find('div', class_='emotion-fq1t0q').findAll('span', class_='emotion-1h6i17m'):
            if category.text not in bad_categories:
                categories.append(category.text)

        #связка с бд, возможно уберу
        helper = DBHelper()
        helper.create_table_simple('category', ['name_cat'])
        helper.create_table('recipe_categories', 'id', 'recipe', 'recipe_id', 'id', ['recipe_id', 'cat_id'], True)

        for category in categories:
            helper.insert('category', ['name_cat'], [category])
            helper.insert('recipe_categories', ['recipe_id', 'cat_id'], [id, category])

        return categories

    #
    def __get_calories(self, id):
        calories = []

        for j in range(4):
            calories.append(self.html.findAll('div', class_='emotion-8fp9e2')[j].text)

        #связка с бд, возможно уберу
        helper = DBHelper()
        #self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params
        helper.create_table('calories', 'recipe_id', 'recipe', 'recipe_id', 'id', ['calories', 'proteins', 'fats', 'carbohydrates'])
        helper.insert_check('calories', 'recipe_id', id, calories)

        return Calories(calories[0], calories[1], calories[2], calories[3])

    def __get_manual(self):
        manual = ''

        for point in self.html.findAll('span', class_='emotion-1dvddtv'):
            manual += point.text.replace(u'\xa0', u' ') + '\n' + '\n'

        return manual

In [133]:
class Recipe:
    def __init__(self, name_dish, count_portion, cook_time, manual):
        self.name = name_dish
        self.count_portion = count_portion
        self.cook_time = cook_time
        self.manual = manual

In [8]:
class Category:
    def __init__(self, name_cat):
        self.name = name_cat

In [9]:
class Ingredient:
    def __init__(self, name_ingr):
        self.name = name_ingr

In [10]:
class Calories: #можно взять по айдишнику рецепта
    def __init__(self, calories, proteins, fats, carbohydrates):
        self.calories = calories
        self.proteins = proteins
        self.fats = fats
        self.carbohydrates = carbohydrates

In [11]:
class Link:
    def __init__(self, link, photo):
        self.link = link
        self.photo = photo

In [127]:
class DBHelper:

    def __init__(self, db_name = "recipe.db"):
        self.dbname = f"sqlite3:/{db_name}"
        self.conn = sqlite3.connect(db_name)

    #for link, recipe, ingredient and category
    def create_table_simple(self, table_name, params):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params) - 1] + " TEXT "

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY AUTOINCREMENT , {column})"
        print(new_table)
        self.conn.execute(new_table)

        print("Выполнилось")

        self.conn.commit()

    #for calories and recipe_ingredients
    def create_table(self, table_name, primary_key, foreign_table_name, foreign_col, another_foreign_col, params, auto = False):
        column = ""
        for i in range(len(params) - 1):
            column += params[i] + " TEXT, "
        column += params[len(params)-1] + " TEXT "

        foreign_key = f"FOREIGN KEY ({foreign_col})  REFERENCES {foreign_table_name} ({another_foreign_col})"

        new_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({primary_key} INTEGER PRIMARY KEY {('', 'AUTOINCREMENT')[auto == True]} , {column}, {foreign_key})"
        self.conn.execute(new_table)

        print("Выполнилось")

        self.conn.commit()

    def insert_check(self, table_name, primary_key, primary_value, args):
        arguments = f"'{primary_value}', '"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        new_insert = f"INSERT INTO {table_name} SELECT * FROM (SELECT {arguments}) AS tmp " \
                     f"WHERE NOT EXISTS ( SELECT {primary_key} FROM {table_name} WHERE {primary_key} = '{primary_value}'" \
                     f") LIMIT 1"
        self.conn.execute(new_insert)

        print("Выполнилось")

        self.conn.commit()

    def insert(self, table_name, column_args, args):
        arguments = "'"
        for i in range(len(args) - 1):
            arguments += str(args[i]) + "', '"
        arguments += str(args[len(args) - 1]) + "'"
        #print(arguments)

        columns = ""
        for i in range(len(column_args) - 1):
            columns += str(column_args[i]) + ", "
        columns += str(column_args[len(column_args) - 1])

        new_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({arguments})"
        print(new_insert)

        #try:
        self.conn.execute(new_insert)

        print("Выполнилось")

        self.conn.commit()
        #except

    def print_info(self, table_name):
        new_get = f"SELECT * FROM {table_name}"
        cursor = self.conn.execute(new_get)

        print("Выполнилось")

        return [row for row in cursor]

    def get(self, table_name, column_args, args):
        query = ""
        for i in range(len(column_args) - 1):
            query += column_args[i] + " = '" + args[i] + "' AND "
            print(query)
        query += column_args[len(column_args) - 1] + " = '" + args[len(column_args) - 1] + "'"

        new_get = f"SELECT * FROM {table_name} WHERE {query}"
        cursor = self.conn.execute(new_get)

        print("Выполнилось")

        return [row for row in cursor]

    #def update(self, flat_id, updates):


    #def delete(self, flat_id):


In [13]:
linkers = CrawlerEda().get_links(1, 5)
print(len(linkers))
for i in linkers:
    print(i.id, i.link, i.photo)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B6A813+48355]
	(No symbol) [0x00AFC4B1]
	(No symbol) [0x00A05358]
	(No symbol) [0x009ED293]
	(No symbol) [0x00A4E37B]
	(No symbol) [0x00A5C473]
	(No symbol) [0x00A4A536]
	(No symbol) [0x00A282DC]
	(No symbol) [0x00A293DD]
	GetHandleVerifier [0x00DCAABD+2539405]
	GetHandleVerifier [0x00E0A78F+2800735]
	GetHandleVerifier [0x00E0456C+2775612]
	GetHandleVerifier [0x00BF51E0+616112]
	(No symbol) [0x00B05F8C]
	(No symbol) [0x00B02328]
	(No symbol) [0x00B0240B]
	(No symbol) [0x00AF4FF7]
	BaseThreadInitThunk [0x76277D59+25]
	RtlInitializeExceptionChain [0x77E9B74B+107]
	RtlClearBits [0x77E9B6CF+191]


In [ ]:
DBHelper("recipe.db").create_table("link", ["link", "photo"])

In [ ]:
DBHelper("recipe.db").print_info("link")

In [ ]:
for j in range(1):
    linkers = CrawlerEda().get_links(700, 1000)
    print(len(linkers))
    for i in linkers:
        DBHelper().insert("link", "id", i.id, [i.link, i.photo])

In [138]:
#попытка получить рецепт
ParserEda(CrawlerEda().get_card_info('https://eda.ru/recepty/zavtraki/zavtrak-slovenskiy-91513')).get_recipe()

get_recipe
Завтрак Словенский
25 минут
CREATE TABLE IF NOT EXISTS recipe (id INTEGER PRIMARY KEY AUTOINCREMENT , name_dish TEXT, count_portion TEXT, cook_time TEXT, manual TEXT )
Выполнилось
INSERT INTO recipe (name_dish, count_portion, cook_time, manual) VALUES ('Завтрак Словенский', '1', '25 минут', 'Колбаску обжариваем на гриле

Картофель обжариваем во фритюре

Яйцо жарим в яичницу

Из самуна, обжаренного бекона, сыра, кетчупа и горчицы делаем тост.

')
Выполнилось
name_dish = 'Завтрак Словенский' AND 
Выполнилось
CREATE TABLE IF NOT EXISTS category (id INTEGER PRIMARY KEY AUTOINCREMENT , name_cat TEXT )
Выполнилось
Выполнилось
INSERT INTO category (name_cat) VALUES ('Завтраки')
Выполнилось
INSERT INTO recipe_categories (recipe_id, cat_id) VALUES ('1', 'Завтраки')
Выполнилось
INSERT INTO category (name_cat) VALUES ('Югославская кухня')
Выполнилось
INSERT INTO recipe_categories (recipe_id, cat_id) VALUES ('1', 'Югославская кухня')
Выполнилось
CREATE TABLE IF NOT EXISTS ingredient (id